# SCPN

This notebook gives a toy example to show how to solve the missing traffic data imputation problem using a low-rank tensor completion approach based on the tensor Schatten capped p norm (SCPN), a flexible and non-convex rank surrogate. 

Note that the parameters used in this example may not necessarily optimal. For an in-depth discussion of SCPN, please refer to our paper.

## Preparations

In [1]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(tensor_size[index]), order = 'F'), 0, mode)

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mae(var, var_hat):
    return np.sum(np.abs(var-var_hat)) / var.shape[0]


def Gradient_Descent(lamda, p, sigma, Iter=10):
    '''
    Parameters:
        lamda: 1/rho
        p: hyper-parameter p
        sigma: the i-th singular value of matrix G
        Iter: the number of iterations in solving x*
    Output:
        x*: float type
    '''
    x_k = sigma
    for k in range(Iter):
        x_k1 = sigma - lamda * p * (x_k) ** (p - 1)
        x_k = x_k1

    return x_k

def opt_x(G, lamda, p, tau):
    '''
    Parameters:
        G: 2-d matrix
        lamda: 1/rho
        p: hyper-parameter p
        tau: hyper-parameter tau (compared with singular value)
    Output:
        W: transformed G (with singular matrix changed)
    '''

    [Q, sigma, R] = np.linalg.svd(G, full_matrices=0)
    delta = np.zeros(sigma.shape)  # singular vector

    v = (2 * lamda * (1 - p)) ** (1 / (2 - p))
    v1 = v + lamda * p * (v ** (p - 1))

    for i in range(len(sigma)):
        s = sigma[i]
        if s >= v1:
            x_ = Gradient_Descent(lamda, p, s)
        else:
            x_ = 0

        tau_ = ((1 / (2 * lamda)) * ((x_ - s) ** 2) + x_ ** p) ** (1 / p)
        if tau <= tau_:
            delta[i] = s
        else:
            delta[i] = x_

    return np.matmul(np.matmul(Q, np.diag(delta)), R)

## Main function

In [2]:
def SCPN(dense_tensor, sparse_tensor, alpha, rho, p, tau, epsilon, maxiter=100):
    '''
    Parameters:
        dense_tensor: the original tensor (without mask) ∈ (M * I * J)
        sparse_tensor: the observed tensor (with mask) ∈ (M * I * J)
        alpha: the weight for each tensor mode, e.g., [0.3, 0.3, 0.3]
        rho: penalty parameter
        p: hyper-parameter p
        tau: hypter-parameter tau
        epsilon: the threshold for residual
        maxiter: the maximum iterations, default = 100
    Output:
        return a recovered tensor, the same size with dense_tensor
    '''

    dim = np.array(sparse_tensor.shape)  # dim = [M, I, J]

    sparse_matrix = ten2mat(sparse_tensor, 0)  # (M * IJ)
    pos_missing = np.where(sparse_matrix == 0)  # all missing values, including original missing and mask missing, = (M * IJ)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))  # only include mask missing, = (M * I * J)

    # min-max normalize
    min_value = ten2mat(dense_tensor, 0).min(axis=1)
    max_value = ten2mat(dense_tensor, 0).max(axis=1)
    sparse_matrix = (sparse_matrix - min_value[:, np.newaxis]) / (max_value[:, np.newaxis] - min_value[:, np.newaxis])
    sparse_matrix[pos_missing] = 0

    # initialization
    X = np.zeros(np.insert(dim, 0, len(dim)))  # 4-D tensor
    T = np.zeros(np.insert(dim, 0, len(dim)))  # 4-D tenosr
    Z = sparse_matrix.copy()  # (M * IJ)
    last_tensor = sparse_tensor.copy()
    it = 0

    # update iteratively
    while True:
        rho = min(rho * 1.4, 1e5)

        # update X 4-D tensor
        for k in range(len(dim)):
            X[k] = mat2ten(
                opt_x(ten2mat(mat2ten(Z, dim, 0) - T[k] / rho, k), 1 / rho, p, tau[k]),
                dim,
                k)

        # update Z matrix (M, IJ)
        Z[pos_missing] = ten2mat(np.mean(X + T / rho, axis=0), 0)[pos_missing]

        # update T 4-D tensor
        T = T + rho * (X - np.broadcast_to(mat2ten(Z, dim, 0), np.insert(dim, 0, len(dim))))

        # predict
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)  # weighted average

        # denormalize
        tensor_hat = mat2ten(ten2mat(tensor_hat, 0) * (max_value[:, np.newaxis] - min_value[:, np.newaxis]) + min_value[:, np.newaxis],dim, 0)

        # check tolerance
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / np.sqrt(np.sum(last_tensor ** 2))
        last_tensor = tensor_hat.copy()
        it += 1

        # terminate
        if (tol < epsilon) or (it >= maxiter):
            break

    # accuracy
    print('Total iteration: {}'.format(it + 1))
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation MAE: {:.6}'.format(compute_mae(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()

    return tensor_hat

## A toy example on PEMS speed dataset
### load data

In [3]:
data = np.load('dataset/PEMS/pems.npy')

dense_tensor = data.reshape(-1,44,288)
print(dense_tensor.shape)
dim = dense_tensor.shape
np.random.seed(1)

(228, 44, 288)


### Random missing scenarios

In [5]:
print('Random missing')
for missing_rate in [0.2, 0.4, 0.6, 0.8]:
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)
    alpha = np.ones(3) / 3
    if missing_rate in [0.4, 0.6]:
        p = 0.5
        lamda = 1e5
        tau = [10, 10, 10]
    elif missing_rate == 0.2:
        p = 0.3
        lamda = 1e3
        tau = [10, 10, 10]        
    else:
        p = 0.5
        lamda = 1e4
        tau = [10, 10, 10]
    rho = 1/lamda
    epsilon = 1e-3
    maxiter = 100
    print('Random missing rate = %.1f' % missing_rate)
    tensor_hat = SCPN(dense_tensor, sparse_tensor, alpha, rho, p, tau, epsilon, maxiter)

Random missing
Random missing rate = 0.2
Total iteration: 24
Imputation MAPE: 0.0232983
Imputation RMSE: 1.70984
Imputation MAE: 1.08583

Random missing rate = 0.4
Total iteration: 38
Imputation MAPE: 0.0276519
Imputation RMSE: 2.03577
Imputation MAE: 1.27003

Random missing rate = 0.6
Total iteration: 38
Imputation MAPE: 0.0364047
Imputation RMSE: 2.63004
Imputation MAE: 1.63696

Random missing rate = 0.8
Total iteration: 31
Imputation MAPE: 0.0550508
Imputation RMSE: 3.82933
Imputation MAE: 2.3944



### Nonrandom missing scenarios

In [6]:
print('Nonrandom missing')
for missing_rate in [0.2, 0.4, 0.6, 0.8]:
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1]) + 0.5 - missing_rate)[:,:,None]
    alpha = np.ones(3) / 3
    if missing_rate in [0.4, 0.6]:
        p = 0.3
        lamda = 1e5
        tau = [20, 20, 20]
    elif missing_rate == 0.8:
        p = 0.5
        lamda = 1e4
        tau = [100, 100, 100]
    else:
        p = 0.1
        lamda = 1e5
        tau = [20, 20, 20]
    rho = 1/lamda
    epsilon = 1e-3
    maxiter = 100
    print('Random missing rate = %.1f' % missing_rate)
    tensor_hat = SCPN(dense_tensor, sparse_tensor, alpha, rho, p, tau, epsilon, maxiter)

Nonrandom missing
Random missing rate = 0.2
Total iteration: 38
Imputation MAPE: 0.0687134
Imputation RMSE: 4.93988
Imputation MAE: 2.96221

Random missing rate = 0.4
Total iteration: 38
Imputation MAPE: 0.0743568
Imputation RMSE: 5.25669
Imputation MAE: 3.16912

Random missing rate = 0.6
Total iteration: 39
Imputation MAPE: 0.0879885
Imputation RMSE: 5.96118
Imputation MAE: 3.66279

Random missing rate = 0.8
Total iteration: 33
Imputation MAPE: 0.107021
Imputation RMSE: 6.95984
Imputation MAE: 4.33058



## License

This work is released under the MIT license.